In [68]:

import pandas as pd
import numpy as np
import os
import glob



# enter period and also enter manual entries in myml as required... e.g maintennace etc
userInput = '2019-09'
myml = (['2019-10-15','2019-10-15','2019-10-15'],
         ['oven 1','oven 2','oven 3'],
         [200,200,400])


# pick up all offset files
os.chdir("H:/SharedLaptop/Pribadi/CBAStatements/CBA Offset")
extension = 'csv'
offset_file = [i for i in glob.glob('*.{}'.format(extension))]
offset_csv = pd.concat([pd.read_csv(f, header=None) for f in offset_file ])
offset_csv['Source'] = 'Offset'

# pick up all card statements
os.chdir("H:/SharedLaptop/Pribadi/CBAStatements/CBA Card")
card_file = [i for i in glob.glob('*.{}'.format(extension))]
card_csv = pd.concat([pd.read_csv(f, header=None) for f in card_file ])
card_csv['Source'] = 'Card'

offset_csv = offset_csv.drop([3],axis=1)
df = pd.concat([offset_csv,card_csv],ignore_index=True)
df.columns = ['Date','Amount','Description','Source']
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)




# replace some descriptions to other to help grouping and remove conflicts
df['Description'] = df['Description'].str.replace('Transfer to other Bank NetBank Admiral','Weekly - The Ponds ')
df['Description'] = df['Description'].str.replace('Bpay','B-Pay', case = False)
df.loc[df['Description'].str.contains('weekly pocket',case=False),'Description'] = 'Weekly Pocket Money'
df.loc[df['Description'].str.contains('pay cba card',case=False),'Description'] = 'Pay CBA Card'
df.loc[df['Description'].str.contains('deft rent',case=False),'Description'] = 'Deft Rent'
df.loc[df['Description'].str.contains('deft payments',case=False),'Description'] = 'Deft Payments'
df.loc[df['Description'].str.contains('bayside',case=False),'Description'] = 'Bayside Council'
df.loc[df['Description'].str.contains('sydney water',case=False),'Description'] = 'Sydney Water'
#df.loc[df['Description'].str.contains('Transfer to xx0765',case=False),'Description'] = 'Pay CBA Card'




In [69]:
df.head()

,Date,Amount,Description,Source
0,2019-09-30,611.5,Direct Credit 536638 M T Flanagan P L Hills Pr...,Offset
1,2019-09-28,-2000.0,Transfer to other Bank NetBank trf to direct shar,Offset
2,2019-09-27,-75.0,Weekly Pocket Money,Offset
3,2019-09-27,-225.0,Weekly Pocket Money,Offset
4,2019-09-26,3156.8,Direct Credit 421520 RONNY CHRISTIANT Ronny Sa...,Offset


In [70]:


# Put default value first for card before doing grouping.... left over needs to be checked later
df.loc[df['Source']=='Card','Group'] = 'Z.Card - Other'


# grouping parameters... please add to mystr and mygroup accordingly
mystr = (['Salary'],
         ['harcourts','hills prestige','hill'],
         ['repay 778047934','repay 876891821','weekly - the ponds'],
         ['weekly pocket'],
         ['deft payments'],
         ['deft rent'],
         ['slick','carena'],
         ['repay 778047926'],
         ['bayside'],
         ['sydney water'],
         ['westpac life'],
         ['sandra low','krishnan','ultrasound','oral care','sheng ruan','chemist','terry white','pline'],
         ['general ins'],
         ['gio','nrma','aami','service nsw'],
         ['medibank','bupa'],
         ['bp','caltex'],
         ['optus'],
         ['AGL'],
         ['kogan','bing lee','jb','harvey','uniqlo','h&m','kmart','bunnings','target','decathlon',
          'reject shop','baby bunting','hair'],
         ['opal'],
         ['netflix'],
         ['tpg'],
         ['coles','woolworths','randwick oriental','eastgate','harris farm'],
         ['top one','super cheap'],
         ['ymca'],
         ['helping hand'],
         ['srg'],
         ['woollahra ps'],
         ['medan ciak','churchill','golden','malatang','shalom','guzman','pork','tonkatsu','chinese','grilld'
         ,'star bar','master wong','my delight','mumbai','mcdonald','tea','hot','japan','hoyts','groupon'
         ,'compass wbc','28 degrees','tonkotsuya','lucky charm','mappen','myeong'],
         ['interesting','virgin','nutrition'],
         ['transfer from'],
         ['transfer to'],
         ['payment received'],
         ['b-pay'],
        )


mygrp = (['A. Salary'],
         ['B. Property - The Ponds Income'],
         ['B. Property - The Ponds Int'],
         ['B. Pocket Money'],
         ['B. Property - Wolli Strata'],
         ['B. Property - Bondi Rent'],
         ['B. Property - Wolli Income'],
         ['B. Property - Wolli Int'],
         ['B. Property - Wolli Council'],
         ['B. Property - Wolli Water'],
         ['C. Insurance - Life'],
         ['C1. Health'],
         ['C. Insurance - Properties'],
         ['C. Insurance - car'],
         ['C. Insurance - Health'],
         ['D. Petrol'],
         ['D. Mobile & Fetch'],
         ['D. Gas & Electricity'],
         ['D. Shopping'],
         ['D. Opal Card'],
         ['D. Netflix'],
         ['D. Internet'],
         ['D. Groceries'],
         ['E. Car - Service'],
         ['Phoebe - Swimming'],
         ['Phoebe - After Care'],
         ['Phoebe - Muaythai'],
         ['Phoebe - School'],
         ['Rest & Entertainment'],
         ['Ronny - Sport'],
         ['Z. Transfer From'],
         ['Z. Transfer To'],
         ['Z. Card Payment'],
         ['Z. Bpay Payment'])


# apply the grouping
for k in range(len(mygrp)):
    count = len(mystr[k])
    for s in range(count):
        df.loc[df['Description'].str.contains(mystr[k][s],case=False),'Group'] = mygrp[k]
        

# create period using string format 'YYYY-MM'        
df['Period'] = df['Date'].dt.strftime("%Y-%m")

# filter the last 6 months data only
#df1 = df1[(df1.Date > '2019-09-30') & (df1.Date < '2020-11-01')] 


# create report
actual = pd.pivot_table(df, values='Amount', index=['Group'],
                    columns=['Period'], aggfunc=np.sum)



In [71]:

# update the list with your forcast numbers
# format is fr, to, freqDate, offsetDate, amt, description

myval = (['2019-10-10','2021-01-31','14D',0,3156,'A. Salary'],
         ['2019-10-01','2019-10-31','14D',0,2103,'A. Salary'],
         ['2019-09-27','2021-01-31','7D',0,-300,'B. Pocket Money'],
         ['2019-09-19','2021-01-31','14D',0,-1300,'B. Property - Bondi Rent'],
         ['2019-08-26','2021-01-31','7D',0,650,'B. Property - The Ponds Income'],
         ['2019-09-25','2021-01-31','7D',0,-889.85,'B. Property - The Ponds Int'],
         ['2019-10-01','2021-01-31','QS-Aug',0,-327.29,'B. Property - Wolli Council'],
         ['2019-09-24','2021-01-31','7D',0,520,'B. Property - Wolli Income'],
         ['2019-09-01','2019-10-08','MS',7,-1145.19,'B. Property - Wolli Int'],
         ['2019-10-09','2021-01-31','7D',0,-311,'B. Property - Wolli Int'],
         ['2019-09-01','2021-01-31','QS-Sep',0,-1200,'B. Property - Wolli Strata'],
         ['2019-10-01','2021-01-31','QS-Oct',0,-173.25,'B. Property - Wolli Water'],
         ['2019-10-01','2020-02-28','MS',0,-421.45,'C. Insurance - Health'],
         ['2020-02-28','2021-02-28','MS',0,-150.45,'C. Insurance - Health'],
         ['2019-09-01','2021-01-31','AS-Sep',0,-870,'C. Insurance - Life'],
         ['2019-09-01','2021-01-31','AS-Sep',0,-1850,'C. Insurance - car'],
         ['2019-10-01','2021-01-31','MS',0,-140.26,'C. Insurance - Properties'],
         ['2019-10-01','2020-01-31','MS',0,-600,'C1. Health'],
         ['2019-10-01','2021-01-31','QS-Sep',0,-750,'D. Gas & Electricity'],
         ['2019-10-01','2021-01-31','MS',0,-350,'D. Groceries'],
         ['2019-10-01','2021-01-31','MS',0,-59.99,'D. Internet'],
         ['2019-10-01','2021-01-31','MS',0,-60,'D. Mobile & Fetch'],    
         ['2019-10-01','2021-01-31','MS',0,-17.99,'D. Netflix'], 
         ['2019-10-01','2021-01-31','MS',0,-20,'D. Opal Card'], 
         ['2019-10-01','2021-01-31','MS',0,-75,'D. Petrol'],
         ['2019-10-01','2021-01-31','MS',0,-50,'D. Shopping'],
         ['2019-10-01','2021-01-31','QS-Oct',0,-300,'E. Car - Service'],
         ['2019-09-26','2021-01-31','14D',0,-81.86,'Phoebe - After Care'],
         ['2019-09-26','2021-01-31','QS-Feb',0,-200,'Phoebe - Muay Thai'],
         ['2019-09-26','2021-01-31','QS-Oct',0,-70,'Phoebe - School'],
         ['2019-10-01','2021-01-31','MS',0,-80,'Phoebe - Swimming'],
         ['2019-10-01','2021-01-31','MS',0,-350,'Rest & Entertainment'],
         ['2019-10-09','2021-01-31','14D',0,-47,'Ronny - Sport'],
         ['2018-10-09','2021-01-31','AS-Oct',0,8000,'Tax Return'])
         
         
         
         
# loop all the description and create dataframe 
for i in range(len(myval)):
        fr = myval[i][0]
        to = myval[i][1]
        frq = myval[i][2]
        offset = myval[i][3]
        amt = myval[i][4]
        desc = myval[i][5]
        
        mydate=pd.date_range(fr,to,freq=frq) + pd.DateOffset(days=offset)
        myamt = np.repeat(amt,len(mydate))
        mydesc = desc
        
        if i ==0 : # create the main dataframe the first time
            df1 = pd.DataFrame ({'Date':mydate, 'Desc':mydesc,'Amount':myamt})
        else:      # create tmp and append to main dataframe
            tmp = pd.DataFrame ({'Date':mydate, 'Desc':mydesc,'Amount':myamt})
            df1 = df1.append(tmp)
            
            
            
# insert user manual entries from the input to dataframe

tmp = pd.DataFrame({'Date':myml[0],'Desc':myml[1],'Amount':myml[2]})
tmp['Date']= pd.to_datetime(tmp['Date'])
df1 = df1.append(tmp)         

            
df1['Period'] = df1['Date'].dt.strftime("%Y-%m")

# filter the last 6 months data only

#df1 = df1[(df1.Date > '2019-09-30') & (df1.Date < '2019-10-31')] 


# produce details report
forecast = pd.pivot_table(df1, values='Amount', index=['Desc'],
                    columns=['Period'], aggfunc=np.sum)



In [72]:

a = actual[userInput]
b = forecast[userInput]

remove = ['Z. Transfer To','Z.Card - Other','Z. Bpay Payment','Z. Card Payment','Z. Transfer From']
rpt_actual = a.drop(remove)


remove = ['Tax Return']
rpt_target = b.drop(remove)

In [73]:
pd.DataFrame({'actual':rpt_actual,'target':rpt_target,'var':rpt_actual-rpt_target})

,actual,target,var
A. Salary,10521.44,NaN,NaN
B. Pocket Money,-1200.00,-300.00,-900.00
B. Property - Bondi Rent,-2600.00,-1300.00,-1300.00
B. Property - The Ponds Income,3259.05,3250.00,9.05
B. Property - The Ponds Int,-4053.37,-889.85,-3163.52
B. Property - Wolli Council,NaN,NaN,NaN
B. Property - Wolli Income,562.85,520.00,42.85
B. Property - Wolli Int,-1145.19,-1145.19,0.00
B. Property - Wolli Strata,-1200.00,-1200.00,0.00
B. Property - Wolli Water,NaN,NaN,NaN


In [74]:
actual.loc[:,'2019-07':]

Period,2019-07,2019-08,2019-09
Group,,,
A. Salary,10495.14,13678.24,10521.44
B. Pocket Money,-1200.00,-1500.00,-1200.00
B. Property - Bondi Rent,-2600.00,-2600.00,-2600.00
B. Property - The Ponds Income,3140.25,611.50,3259.05
B. Property - The Ponds Int,-4933.29,-4053.37,-4053.37
B. Property - Wolli Council,NaN,-327.59,NaN
B. Property - Wolli Income,3225.60,537.60,562.85
B. Property - Wolli Int,-1108.25,-1145.19,-1145.19
B. Property - Wolli Strata,NaN,NaN,-1200.00


In [75]:
forecast.loc[:,'2019-10':'2020-03']

Period,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
Desc,,,,,,
A. Salary,12621.00,6312.00,6312.00,9468.00,6312.00,6312.00
B. Pocket Money,-1200.00,-1500.00,-1200.00,-1500.00,-1200.00,-1200.00
B. Property - Bondi Rent,-3900.00,-2600.00,-2600.00,-2600.00,-2600.00,-2600.00
B. Property - The Ponds Income,2600.00,2600.00,3250.00,2600.00,2600.00,3250.00
B. Property - The Ponds Int,-4449.25,-3559.40,-3559.40,-4449.25,-3559.40,-3559.40
B. Property - Wolli Council,NaN,-327.29,NaN,NaN,-327.29,NaN
B. Property - Wolli Income,2600.00,2080.00,2600.00,2080.00,2080.00,2600.00
B. Property - Wolli Int,-2389.19,-1244.00,-1244.00,-1555.00,-1244.00,-1244.00
B. Property - Wolli Strata,NaN,NaN,-1200.00,NaN,NaN,-1200.00


In [76]:
remove = ['Z. Transfer To','Z.Card - Other','Z. Bpay Payment','Z. Card Payment','Z. Transfer From']
actual.loc[remove,:]

Period,2019-07,2019-08,2019-09
Group,,,
Z. Transfer To,-2000.0,-38340.81,-7280.0
Z.Card - Other,-83.8,-79.35,NaN
Z. Bpay Payment,NaN,-881.00,-27.0
Z. Card Payment,500.0,6007.81,5580.0
Z. Transfer From,NaN,30758.00,4054.0


In [77]:

# check the details on those weird categories
df[(df['Group'].isin(['Z.Card - Other','Z. Bpay Payment'])) & (df['Period'].isin([userInput]))]

,Date,Amount,Description,Source,Group,Period
33,2019-09-02,-27.0,COLES MASTERCARD CommBank app B-Pay 8862 5...,Offset,Z. Bpay Payment,2019-09


In [78]:
# query specific description against actual data
df[df['Description'].str.contains('gio',case=False)]

,Date,Amount,Description,Source,Group,Period
130,2019-09-18,-437.16,GIO CTP BRISBANE QLD,Card,C. Insurance - car,2019-09
